In [14]:
import openai
import os, sys 
import json

In [15]:
print(openai)
openai.api_key = "..."

<module 'openai' from '/Users/hima/.pyenv/versions/3.10.4/lib/python3.10/site-packages/openai/__init__.py'>


In [33]:
default_json_output_instructions = """
    Return output as a JSON list inside a code block.
        """


In [16]:
messages = [
        {"role": "system", "content": "You are the distinguished principal staff tech lead engineer manager L10 at Google."},
    ]

In [17]:

def complete_response(message: str):
    MODEL = "gpt-3.5-turbo"
    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=messages + [message],
        temperature=0.3,
    )
    output = response['choices'][0]['message']['content']


    return output

In [24]:
cli_spec_list_raw_output = complete_response({"content": 
"""Generate the CLI commands for a CLI tool that:
- Encodes/decoes base64 strings
- There should be 2 separate commands for encode and decode
- Allow user to change encoding base

Return output as a JSON list. 
e.g. [{"command_name": "...",  [{"option_name": "...", "option_description": "...}]}]
""", 
"role": "user"},)

print(cli_spec_list_raw_output)

Here are the CLI commands for a CLI tool that encodes/decodes base64 strings with the ability to change encoding base:

```
[
  {
    "command_name": "encode",
    "options": [
      {
        "option_name": "-s, --string",
        "option_description": "The string to encode in base64."
      },
      {
        "option_name": "-b, --base",
        "option_description": "The base to use for encoding. Default is base64.",
        "default": "base64"
      }
    ]
  },
  {
    "command_name": "decode",
    "options": [
      {
        "option_name": "-s, --string",
        "option_description": "The base64 string to decode."
      },
      {
        "option_name": "-b, --base",
        "option_description": "The base to use for decoding. Default is base64.",
        "default": "base64"
      }
    ]
  }
]
```

The output is a JSON list with two objects, one for the `encode` command and one for the `decode` command. Each object contains an array of options with their respective names and d

In [25]:
commands_list = """
[
    {
        'command_name': 'encode_base64',
        'options': [
            {
                'option_name': '-s, --string',
                'option_description': 'the string to encode'
            },
            {
                'option_name': '-b, --base',
                'option_description': 'the base to use for encoding (default: base64)'
            }
        ],
        'example': "encode_base64 -s 'Hello, world!' -b base32"
    },
    {
        'command_name': 'decode_base64',
        'options': [
            {
                'option_name': '-s, --string',
                'option_description': 'the string to decode'
            },
            {
                'option_name': '-b, --base',
                'option_description': 'the base to use for decoding (default: base64)'
            }
        ],
        'example': "decode_base64 -s 'SGVsbG8sIHdvcmxkIQ==' -b base32"
    }
]
"""

In [28]:



methods_list_raw_output = complete_response({"content": 
     f"""
     For these CLI commands, write all the method signatures that we would need to implement them:
    {commands_list}

    Return output as a JSON list inside a code block.
    Think step by step first.
    e.g. [{{"method_signature": "..."}}]

""", 
"role": "user"},)
print(methods_list_raw_output)

# methods_list = json.loads(methods_list_string)
# print(methods_list)

[
    {
        "method_signature": "def encode_base64(string: str, base: str = 'base64') -> str:"
    },
    {
        "method_signature": "def decode_base64(string: str, base: str = 'base64') -> str:"
    }
]


In [30]:
def write_method_body_implementation(method_signature_payload):

    body = complete_response({"content": 
        f"""
        For this method signature, write the body implementation:
        {method_signature_payload["method_signature"]}

        Return output inside a code block.

    """, 
    "role": "user"},)
    print(body)

write_method_body_implementation(json.loads(
    """{
        "method_signature": "def encode_base64(string: str, base: str = 'base64') -> str:"
    }"""))

Here's an implementation of the `encode_base64` function using the `base64` module in Python:

```python
import base64

def encode_base64(string: str, base: str = 'base64') -> str:
    if base == 'base64':
        encoded_bytes = base64.b64encode(string.encode('utf-8'))
    elif base == 'base32':
        encoded_bytes = base64.b32encode(string.encode('utf-8'))
    elif base == 'base16':
        encoded_bytes = base64.b16encode(string.encode('utf-8'))
    else:
        raise ValueError('Invalid base specified')
    return encoded_bytes.decode('utf-8')
```

This function takes in a string to encode and an optional `base` parameter indicating the base encoding to use (default is `base64`). It first checks if the specified base is valid, and then uses the appropriate `base64` encoding function to encode the string. Finally, it decodes the resulting bytes into a string and returns it.


In [31]:
method_body = """
```python
import base64

def encode_base64(string: str, base: str = 'base64') -> str:
    if base == 'base64':
        encoded_bytes = base64.b64encode(string.encode('utf-8'))
    elif base == 'base32':
        encoded_bytes = base64.b32encode(string.encode('utf-8'))
    elif base == 'base16':
        encoded_bytes = base64.b16encode(string.encode('utf-8'))
    else:
        raise ValueError('Invalid base specified')
    return encoded_bytes.decode('utf-8')
```
"""

In [37]:
def write_method_test_signatures(method_body_string):

    body = complete_response({"content": 
        f"""
        For this given method implementation, write only descriptions for possible tests:
        {method_body_string}

        {default_json_output_instructions}
        e.g. [{{"test_name": "...", "test_comment": "...}}]

    """, 
    "role": "user"},)
    print(body)

write_method_test_signatures(method_body)

[{"test_name": "test_base64_encoding", "test_comment": "Test if the function correctly encodes a string using base64 encoding"},
 {"test_name": "test_base32_encoding", "test_comment": "Test if the function correctly encodes a string using base32 encoding"},
 {"test_name": "test_base16_encoding", "test_comment": "Test if the function correctly encodes a string using base16 encoding"},
 {"test_name": "test_invalid_base", "test_comment": "Test if the function raises a ValueError when an invalid base is specified"},
 {"test_name": "test_empty_string", "test_comment": "Test if the function correctly encodes an empty string"}]


In [39]:
test_signature = json.loads("""
{"test_name": "test_encode_base64_with_base32", "test_comment": "Test encoding with base32"}
""")

{'test_name': 'test_encode_base64_with_base32', 'test_comment': 'Test encoding with base32'}


In [42]:
def write_method_test_implementation(method_test_signature):

    body = complete_response({"content": 
        f"""
        For this given test info, write only the test implementation:
        - test name: {method_test_signature["test_name"]}
        - test comment: {method_test_signature["test_comment"]}
        
        Return output inside a code block, with language specified.

    """, 
    "role": "user"},)
    print(body)

write_method_test_implementation(test_signature)

```python
def test_encode_base64_with_base32():
    input_str = "Hello, World!"
    expected_output = "JBSWY3DPEBLW64TMMQQQ===="
    actual_output = encode_base64_with_base32(input_str)
    assert actual_output == expected_output, f"Expected {expected_output}, but got {actual_output}"
```


In [ ]:
for method_signature_payload in methods_list:
    method_implementation = write_method_implementation(method_signature_payload)
    test_signatures = write_method_test_signatures(method_implementation)
    
    all_method_tests = []
    for test_sig in test_signatures:
        method_test_implementation = write_method_test_implementation(test_sig)
        all_method_tests.append(method_test_implementation)

    
has_failures=False 
for test in all_method_tests:
    test_result = run_test(test)
    if test_result.status == "failure":
        fix_code()
        test_result = run_test(test)
        # TODO



    


    

